In [ ]:
import os
import cv2
import numpy as np
import time
import yaml
import PIL.Image
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

In [ ]:
# Put numpy array to video formate frame
def video_HDMI(hdmi_out, frame):
    
    # Read input frame size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    #print(frame.shape)
    
    # Creat new video formate frame
    outframe = hdmi_out.newframe()
    
    # Fill new frame with zero
    zero_img = np.zeros((480, 640), dtype=np.uint8)
    outframe[0:480,0:640] = zero_img[0:480,0:640]

    # Transfer array to the video frame
    # For Gray imge
    outframe[0:img_h,0:img_w] = frame[0:img_h,0:img_w]
    # For RBG image
    #outframe[0:480,0:640,:] = edges[0:480,0:640,:]
    
    return outframe

In [ ]:
# Initialize HDMI I/O

# Load the overlay
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure HDMI input to gray scale ( (0.3 * R) + (0.59 * G) + (0.11 * B) )
hdmi_in.configure(PIXEL_GRAY)
hdmi_in.start()

# Configure Output resolution (w, h, bit per pixek)
hdmi_out_mode = VideoMode(640,480,8)
hdmi_out.configure(hdmi_out_mode, PIXEL_GRAY)
hdmi_out.start()

In [ ]:
# Read calibration parameter

fname = "calibration_parameter.yaml"
with open(fname) as file:
    data = yaml.load(file,Loader=yaml.Loader)
    
mtx = np.array([ [ data[0] , 0, data[1] ] , [ 0, data[2], data[3] ] , [0, 0, 1] ])
dist = np.array([ [ data[4], data[5], data[6], data[7], data[8] ] ])

In [ ]:
image_saved = 0 # Track number of images saved

In [ ]:
# Display live video to ensure the subject is within the view
# Stop this block once you confirm you have the desired view
# Run the next block to take picture
while (True):
    
    gray = hdmi_in.readframe()
    
    img = cv2.resize(gray, (640,360))

    # Undistort
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    # crop the image
    x,y,w,h = roi
    
    # Align center mannually
    dst = dst[y:y+h, x+120:x+w-150]
    #print(-20+w-150)
    
    outframe = video_HDMI(hdmi_out, dst)
    
    hdmi_out.writeframe(outframe)

In [ ]:
# This block of code will save one image at a time
# Get image with different angle and distant for the subject
# The image will be stored under the image_path

cwd = os.getcwd() # get current working direction

### parameter ###
image_path = cwd + '/new_pic/' # path to store new user's image

gray = hdmi_in.readframe()

img = cv2.resize(gray, (640,360))

# Undistort
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi

# Align center mannually
dst = dst[y:y+h, x+120:x+w-150]
#print(-20+w-150)


cv2.imwrite(image_path + str(image_saved) + ".bmp", dst) # save image

image_saved = image_saved + 1 # save image counting

print(image_saved)

outframe = video_HDMI(hdmi_out, dst)

hdmi_out.writeframe(outframe)

print(image_path)

In [ ]:
# After taking all picture, stop HMDI ports

hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out